In [ ]:
import numpy as np
import scipy.stats as stats
import scipy as sp
import cupy
import matplotlib.pyplot as plt
%matplotlib widget

sys.path.insert(0, '../../drlib')
import drlib as dr


In [ ]:
def mad_np(arr):
    median = np.median(arr)
    return np.median(np.abs(arr-median))
    

In [ ]:
arr_cpu = np.random.random(2**10)
arr_gpu = cupy.random.random(2**10)

In [ ]:
%%timeit
median = np.median(arr_cpu)
mad_cpu = np.median(np.abs(arr_cpu-median))

In [ ]:
%%timeit
mad_np(arr_cpu)

In [ ]:
%%timeit
mad_scipy = stats.median_abs_deviation(arr_cpu)


In [ ]:
%%timeit
median_gpu = cupy.median(arr_gpu)
mad_gpu = cupy.median(cupy.abs(arr_gpu-median_gpu))

In [ ]:
print('mad cpu =    ', mad_cpu)
print('mad scipy =  ', mad_scipy)
print('mad gpu =    ', mad_gpu)

In [ ]:
#generate test data (x)

#generate spectrum with known STD (sin^2)
testLength      = 2**23
numStdPoints    = 2**10
stds            = 3*np.sin(np.linspace(0, 40*3.14, numStdPoints))**2 + 1
stdsInterp      = 3*np.sin(np.linspace(0, 40*3.14, testLength))**2 + 1
x               = np.random.normal(0, stds,  size=(testLength//numStdPoints, numStdPoints)) 
x               = np.reshape(x.T, testLength)

#Inject spurs
x[int(testLength//2.34)] = 1000

plt.figure()
plt.title("Simulated Spectrum")
plt.plot(x)
plt.plot(np.linspace(0, testLength, numStdPoints), stds * 5, label = "5 * true std (before spurs)")
plt.xlabel('Frequency (arb)')
plt.ylabel('Power (arb)')
plt.legend(loc = 'lower right')

print('mean  = ', x.mean())
print('mad * 1.48 = ', 1.4826*sp.stats.median_abs_deviation(x))
print('std = ', np.std(x))


In [ ]:
window = 1000
def mad_np(arr):
    median = np.median(arr)
    return np.median(np.abs(arr-median))

In [ ]:
#%%timeit
npRollingMad = dr.rolling(x, window, 1, mad_np, numProc=48)

In [ ]:
#%%timeit
spRollingMad = dr.rolling(x, window, 1, sp.stats.median_abs_deviation, numProc=48)

In [ ]:
((npRollingMad-spRollingMad)!=0).sum()

In [ ]:
filter_fc           = 30 * window
filteredMadArr      = dr.filterSpec(npRollingMad, order = 2, fc_numBins = filter_fc, type = 'lowpass')
filteredMadPadArr   = dr.nanPad(filteredMadArr, window)

plt.close('all')
plt.figure()
plt.title("Simulated Spectrum")
#plt.plot(x)
plt.plot(np.linspace(0, testLength, numStdPoints), stds * 5, label = "5 * true std (before spurs)")
#plt.plot(npRollingMad*5*1.48, label = "Rolling Mad")
plt.plot(filteredMadArr*5*1.48, label = "Filtered Rolling Mad")
plt.xlabel('Frequency (arb)')
plt.ylabel('Power (arb)')
plt.legend(loc = 'lower right')

plt.figure()
plt.plot(filteredMadPadArr - stdsInterp)
plt.plot(np.linspace(0, testLength, numStdPoints), stds )

Old Stuff from run 1 analysis notebook


In [ ]:
#mp spcipy based homebrew 
#print(x)
window = 5000
step = 1
spec = x

strideSpec = np.lib.stride_tricks.sliding_window_view(spec,window)

#multiprocess rolling MAD on strided array
def MAD(spec):
    #print(spec)
    #return np.full(step, sp.stats.median_abs_deviation(spec))
    return sp.stats.median_abs_deviation(spec)
with mp.Pool(40) as p:
    rollingMadMp = p.map(MAD, strideSpec[::step])

rollingMadMp = np.asarray(rollingMadMp).reshape(-1)

'''
infArr = np.full(window//2, float('nan'))
rollingMadMp = np.append(infArr, rollingMadMp)
rollingMadMp = np.append(rollingMadMp, infArr)
'''


In [ ]:
window = 3
step = 2
spec = x
strideSpec = np.lib.stride_tricks.sliding_window_view(spec,window)
rollingMadMp = np.zeros(len(spec - window))

for i, spec in enumerate(strideSpec[::step]):
    rollingMadMp[i] = sp.stats.median_abs_deviation(spec)


In [ ]:
#rolling std using pd rolling
antSpecFiltered = dr.filterSpec(antSpec)
termSpecFiltered = dr.filterSpec(termSpec)



antFilter_Df = pd.DataFrame(antSpecFiltered)
antRollingStd = antFilter_Df.rolling(10000).std()
termFilter_Df = pd.DataFrame(termSpecFiltered)
termRollingStd = termFilter_Df.rolling(10000).std()
termRollingMad = rollingMadMp

numTermCandidates = (termSpecFiltered > (6*termRollingStd).to_numpy().T[0]).sum()
numAntCandidates = (antSpecFiltered > (6*antRollingStd).to_numpy().T[0]).sum()
print(numTermCandidates, 'candidates on terminator')
print(numAntCandidates, 'candidates on antenna')

plt.figure()
plt.title('antenna')
plt.plot(antSpecFiltered)
plt.plot(antRollingStd*5)

plt.figure()
plt.title('terminator')
plt.plot(termSpecFiltered)
plt.plot(termRollingStd*5)
plt.plot(termRollingMad*5)

plt.figure()
plt.plot(antSpec)
plt.plot(termSpec)


In [ ]:
plt.figure()
plt.plot(rollingMad)
plt.plot(rollingStd * .67)

In [ ]:
#pandas rolling std

df = pd.DataFrame(antSpecCleanFilter)
arr = np.asarray(df.rolling(10000).std())
limit = 6 * arr

plt.figure()
plt.plot(freqs, antSpecCleanFilter)
plt.plot(freqs, limit, label = '5sigma thresh')

'''
count = 0
for i in range(len(antSpecCleanFilter)):
    if antSpecCleanFilter[i]>limit[i]:
        count += 1
print(count)
'''

In [ ]:
df = pd.DataFrame(antSpec)
arr = (df.rolling(100).median())
diff = (df - arr)
std = 5 * diff.rolling(10000).std()

plt.figure()
plt.plot(freqs, antSpec)
plt.plot(freqs,arr)

plt.figure()
plt.plot(freqs, diff)
plt.plot(freqs, std)
